In [1]:
# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text    

# !pip install --user tensorflow_text


# !pip install tensorflow_hub

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text    


import pandas as pd

df = pd.read_csv('Data.csv')
# "C:\Users\akash\Python_DL\project_data\Data.csv"
df.head(5)

df.groupby('Category').describe()

2023-08-01 11:09:20.511285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 11:09:33.394235: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [2]:
# Separate data frames for individual classes (Class A and Class B)
spam = df[df['Category'] == 'spam']
ham = df[df['Category'] == 'ham']

# Display the data frames
print("DataFrame for Class spam:")
print(spam)

print("\nDataFrame for Class ham:")
print(ham)


DataFrame for Class spam:
     Category                                            Message
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
5        spam  FreeMsg Hey there darling it's been 3 week's n...
8        spam  WINNER!! As a valued network customer you have...
9        spam  Had your mobile 11 months or more? U R entitle...
11       spam  SIX chances to win CASH! From 100 to 20,000 po...
...       ...                                                ...
5537     spam  Want explicit SEX in 30 secs? Ring 02073162414...
5540     spam  ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547     spam  Had your contract mobile 11 Mnths? Latest Moto...
5566     spam  REMINDER FROM O2: To get 2.50 pounds free call...
5567     spam  This is the 2nd time we have tried 2 contact u...

[747 rows x 2 columns]

DataFrame for Class ham:
     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham 

In [3]:
# Downsample Class B to have the same number of samples as Class A
ham_downsampled = ham.sample(n=len(spam), random_state=42)

# Display the downsampled DataFrame for Class B
print("DataFrame for Class ham (Downsampled):")
print(ham_downsampled)

DataFrame for Class ham (Downsampled):
     Category                                            Message
3714      ham  If i not meeting ü all rite then i'll go home ...
1311      ham  I.ll always be there, even if its just in spir...
548       ham                   Sorry that took so long, omw now
1324      ham  I thk 50 shd be ok he said plus minus 10.. Did...
3184      ham  Dunno i juz askin cos i got a card got 20% off...
...       ...                                                ...
4992      ham  We made it! Eta at taunton is 12:30 as planned...
3117      ham                Uncle Abbey! Happy New Year. Abiola
4975      ham  Aiyo u so poor thing... Then u dun wan 2 eat? ...
3505      ham                          Will you be here for food
1983      ham  Hey i will be late... i'm at amk. Need to drin...

[747 rows x 2 columns]


In [4]:
all_downsampled = pd.concat([spam, ham_downsampled], ignore_index=True)

# Display the concatenated DataFrame
print("Concatenated DataFrame:")
print(all_downsampled)

Concatenated DataFrame:
     Category                                            Message
0        spam  Free entry in 2 a wkly comp to win FA Cup fina...
1        spam  FreeMsg Hey there darling it's been 3 week's n...
2        spam  WINNER!! As a valued network customer you have...
3        spam  Had your mobile 11 months or more? U R entitle...
4        spam  SIX chances to win CASH! From 100 to 20,000 po...
...       ...                                                ...
1489      ham  We made it! Eta at taunton is 12:30 as planned...
1490      ham                Uncle Abbey! Happy New Year. Abiola
1491      ham  Aiyo u so poor thing... Then u dun wan 2 eat? ...
1492      ham                          Will you be here for food
1493      ham  Hey i will be late... i'm at amk. Need to drin...

[1494 rows x 2 columns]


In [5]:
label_mapping = {'spam': 1, 'ham': 0}
all_downsampled['Label'] = all_downsampled['Category'].apply(lambda x: label_mapping[x])

# Display the updated DataFrame with the 'Label' column
print(all_downsampled)

     Category                                            Message  Label
0        spam  Free entry in 2 a wkly comp to win FA Cup fina...      1
1        spam  FreeMsg Hey there darling it's been 3 week's n...      1
2        spam  WINNER!! As a valued network customer you have...      1
3        spam  Had your mobile 11 months or more? U R entitle...      1
4        spam  SIX chances to win CASH! From 100 to 20,000 po...      1
...       ...                                                ...    ...
1489      ham  We made it! Eta at taunton is 12:30 as planned...      0
1490      ham                Uncle Abbey! Happy New Year. Abiola      0
1491      ham  Aiyo u so poor thing... Then u dun wan 2 eat? ...      0
1492      ham                          Will you be here for food      0
1493      ham  Hey i will be late... i'm at amk. Need to drin...      0

[1494 rows x 3 columns]


In [6]:
from sklearn.model_selection import train_test_split



# Features (X) are all the columns except 'Category' and 'Label'
X = all_downsampled.drop(columns=['Category', 'Label'])

# Labels (y) are contained in the 'Label' column
y = all_downsampled['Label']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print("Training set shape - X_train:", X_train.shape, "y_train:", y_train.shape)
print("Testing set shape - X_test:", X_test.shape, "y_test:", y_test.shape)

Training set shape - X_train: (1195, 1) y_train: (1195,)
Testing set shape - X_test: (299, 1) y_test: (299,)


In [7]:
X_train.head()

,Message
532,Bored housewives! Chat n date now! 0871750.77....
534,This is the 2nd time we have tried to contact ...
1108,ALRITE
490,Promotion Number: 8714714 - UR awarded a City ...
933,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
# Downloading the bert models for pre processing and encoding


bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2023-08-01 11:10:12.731347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
38/38 [==============================] - 467s 12s/step - loss: 0.6734 - accuracy: 0.5900
Epoch 2/10
38/38 [==============================] - 445s 12s/step - loss: 0.5270 - accuracy: 0.7983
Epoch 3/10
38/38 [==============================] - 451s 12s/step - loss: 0.4473 - accuracy: 0.8619
Epoch 4/10
38/38 [==============================] - 628s 17s/step - loss: 0.3988 - accuracy: 0.8753
Epoch 5/10
38/38 [==============================] - 683s 18s/step - loss: 0.3629 - accuracy: 0.8854
Epoch 6/10
38/38 [==============================] - 312s 8s/step - loss: 0.3366 - accuracy: 0.8946
Epoch 7/10
38/38 [==============================] - 409s 11s/step - loss: 0.3185 - accuracy: 0.8996
Epoch 8/10
38/38 [==============================] - 393s 10s/step - loss: 0.2941 - accuracy: 0.9105
Epoch 9/10
38/38 [==============================] - 415s 11s/step - loss: 0.2917 - accuracy: 0.9155
Epoch 10/10
38/38 [==============================] - 403s 11s/step - loss: 0.2787 - accuracy: 0.9071


In [13]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 96s 9s/step - loss: 0.3127 - accuracy: 0.8896


[0.312650591135025, 0.8896321058273315]

In [14]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]


model.predict(reviews)


1/1 [==============================] - 2s 2s/step


array([[0.83589005],
       [0.87188363],
       [0.84463155],
       [0.2289247 ],
       [0.13511556]], dtype=float32)

In [15]:
model.save('trail_1', include_optimizer=False)

INFO:tensorflow:Assets written to: trail_1/assets


INFO:tensorflow:Assets written to: trail_1/assets


In [16]:
reloaded_model = tf.saved_model.load('spam_text_classifier')

OSError: SavedModel file does not exist at: spam_text_classifier/{saved_model.pbtxt|saved_model.pb}

In [ ]:
reloaded_results = tf.sigmoid(reloaded_model(reviews))

In [ ]:
print('Results from the saved model:')
type(reloaded_results)

In [ ]:
reloaded = reloaded_results.numpy()

In [ ]:
reloaded[0][0]

In [ ]:
raw = [
    'Get assured prize for helping Tom in MI Challenge ',
    'Everything you need for outrageously effective odor control',
    '5 minutes. 250+ Data projects. 1-1 session with an advisor',
    'Nah, I dont think',
]

In [ ]:
model.predict(raw)